<h3 align="center" style="margin-top:0">Lab - EDA Bivariate Analysis: Diving into Amazon UK Product Insights Part II </h1>

<p align="center">
  <img src="amazon-uk-logo-feature.jpg" alt="Lab - EDA Univariate Analysis: Diving into Amazon UK Product Insights" width="440">
</p>

<p align="center"><em>EDA Univariate Analysis:</em></p>

<p align="center">
  <strong> David Hernández
</p>

<p align="center">
  <strong> November 2025
</p>

<hr style="margin:18px 0">

## 1. Introduction

**Objective:** Delve into the dynamics of product pricing on Amazon UK to uncover insights that can inform business strategies and decision-making.

**Dataset**: This lab utilizes the Amazon UK product dataset which provides information on product categories, brands, prices, ratings, and more from from Amazon UK. You'll need to download it to start working with it.

Data Source: https://www.kaggle.com/datasets/asaniczka/uk-optimal-product-price-prediction/

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import numpy as np
import scipy.stats as stats

In [ ]:
df = pd.read_csv("C:/Users/jdhernandezd/Documents/IRONHACK/Labs/lab-eda-bivariate/amz_uk_price_prediction_dataset.csv")
df.head()

##### Part 1: Analyzing Best-Seller Trends Across Product Categories

In [ ]:
# Crosstab Analysis
crosstab = pd.crosstab(df['category'], df['isBestSeller'])
crosstab


In [ ]:
# Chi-Squared Test
# from scipy.stats import chi2_contingency

chi2, p, dof, expected = chi2_contingency(crosstab)

chi2, p, dof


In [ ]:
# Cramér's V
# import numpy as np

n = crosstab.sum().sum()
cramers_v = np.sqrt(chi2 / (n * (min(crosstab.shape)-1)))

cramers_v

# Scale for interpretation:
# 0.0 – 0.1  Very weak
# 0.1 – 0.2  Weak
# 0.2 – 0.3  Moderate
# 0.3+  Strong


In [ ]:
# Stacked Bar Chart

crosstab_norm = pd.crosstab(df['category'], df['isBestSeller'], normalize='index')

crosstab_norm.plot(
    kind='bar',
    stacked=True,
    figsize=(12,6),
    colormap='viridis'
)

plt.title("Best-Seller Proportion by Product Category")
plt.xlabel("Category")
plt.ylabel("Proportion")
plt.legend(title="Is Best Seller", labels=["No", "Yes"])
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()
# Need to limit to top categories

In [ ]:
best_seller_ratio = (
    pd.crosstab(df['category'], df['isBestSeller'], normalize='index')[True]
    .sort_values(ascending=False)
)

best_seller_ratio.head(10)


In [ ]:
top10 = best_seller_ratio.head(10).index

subset = df[df['category'].isin(top10)]

crosstab_top10 = pd.crosstab(subset['category'], subset['isBestSeller'], normalize='index')

crosstab_top10.plot(
    kind='bar',
    stacked=True,
    figsize=(10,6),
    colormap='viridis'
)

plt.title("Best-Seller Proportion — Top 10 Categories")
plt.xlabel("Category")
plt.ylabel("Proportion")
plt.xticks(rotation=45, ha="right")
plt.tight_layout()
plt.show()


###  Business Summary — Analyzing Best-Seller Trends Across Product Categories

- Product category does have a statistically significant impact on whether a product becomes a best-seller.
- However, the strength of this relationship is weak (Cramér’s V = 0.122).
- Even in the top-performing categories, best-sellers represent only a small proportion of all listings.
- This suggests that success on Amazon UK depends on factors beyond just category—such as brand, price positioning, product quality, marketing, and reviews.


Part 2: Exploring Product Prices and Ratings Across Categories and Brands

Objective: Investigate how different product categories influence product prices.

In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

lower_bound, upper_bound

In [ ]:
df_no_outliers = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]
df_no_outliers.shape


In [ ]:
top20_categories = df_no_outliers['category'].value_counts().head(20).index

In [ ]:
df_top20 = df_no_outliers[df_no_outliers['category'].isin(top20_categories)]


In [ ]:
plt.figure(figsize=(14, 8))
sns.violinplot(
    data=df_top20,
    x='category',
    y='price',
    inner='quartile'
)
plt.xticks(rotation=45, ha='right')
plt.title("Price Distribution Across Top 20 Product Categories")
plt.tight_layout()
plt.show()


In [ ]:
median_prices = df_no_outliers.groupby('category')['price'].median().sort_values(ascending=False)
median_prices.head(10)


In [ ]:
top10_categories = df_no_outliers['category'].value_counts().head(10).index


In [ ]:
df_top10 = df_no_outliers[df_no_outliers['category'].isin(top10_categories)]


In [ ]:
avg_prices = df_top10.groupby('category')['price'].mean().sort_values(ascending=False)
avg_prices


In [ ]:
plt.figure(figsize=(12,6))
sns.barplot(
    data=df_top10,
    x='category',
    y='price',
    estimator='mean',
    order=avg_prices.index
)
plt.xticks(rotation=45, ha='right')
plt.title("Average Price for Top 10 Product Categories (by count)")
plt.xlabel("Category")
plt.ylabel("Average Price")
plt.tight_layout()
plt.show()


In [ ]:
avg_prices_global = df_no_outliers.groupby('category')['price'].mean().sort_values(ascending=False)
avg_prices_global.head(10)


In [ ]:
top10_categories = df_no_outliers['category'].value_counts().head(10).index


In [ ]:
df_top10 = df_no_outliers[df_no_outliers['category'].isin(top10_categories)]


In [ ]:
plt.figure(figsize=(12,7))
sns.boxplot(
    data=df_top10,
    x='category',
    y='stars'
)
plt.xticks(rotation=45, ha='right')
plt.title("Product Ratings Distribution — Top 10 Categories")
plt.xlabel("Category")
plt.ylabel("Rating (Stars)")
plt.tight_layout()
plt.show()


In [ ]:
median_ratings = df_no_outliers.groupby('category')['stars'].median().sort_values(ascending=False)
median_ratings.head(10)


### Business Summary — Exploring Product Prices Across Categories

- Product prices vary substantially across categories, even after removing outliers using the IQR method.

- Some categories consistently command higher prices, with electronics-related categories (such as Motherboards) showing the highest median and average price levels.

- This indicates that certain product groups occupy more premium segments within Amazon UK, likely due to higher production costs, technical complexity, or strong consumer demand.

- Other categories maintain lower and more compact price ranges, suggesting highly competitive or low-margin product spaces.

- Overall, price differences are strongly influenced by category, highlighting the importance for sellers to understand category-specific pricing patterns when positioning their products.

Part 3: Investigating the Interplay Between Product Prices and Ratings

Objective: Analyze how product ratings (stars) correlate with product prices.

In [ ]:
corr_price_rating = df_no_outliers['price'].corr(df_no_outliers['stars'])
corr_price_rating

In [ ]:
plt.figure(figsize=(8,6))
sns.scatterplot(
    data=df_no_outliers,
    x='stars',
    y='price',
    alpha=0.3
)
plt.title("Relationship Between Product Rating and Price")
plt.xlabel("Rating (Stars)")
plt.ylabel("Price")
plt.show()


In [ ]:
numeric_df = df_no_outliers.select_dtypes(include=['number'])
numeric_df.head()


In [ ]:
plt.figure(figsize=(10,6))
sns.heatmap(
    numeric_df.corr(),
    annot=True,
    cmap='coolwarm',
    fmt=".2f"
)
plt.title("Correlation Heatmap — Numerical Variables")
plt.show()


In [ ]:
#import scipy.stats as stats
#import matplotlib.pyplot as plt

plt.figure(figsize=(6,6))

stats.probplot(df_no_outliers['price'], dist="norm", plot=plt)

plt.title("QQ Plot — Price Normality Check")
plt.show()


### Business Summary — Investigating the Interplay Between Product Prices and Ratings

- The QQ plot shows that product prices do not follow a normal distribution.
- The S-shaped pattern indicates strong right skew and heavy tails, which is typical of real-world e-commerce pricing. Even after removing outliers using the IQR method, the distribution remains non-normal.